In [1]:
import numpy as np
import math

In [3]:
def euler2A(fi, teta, ksi):
    cos_fi = math.cos(fi)
    sin_fi = math.sin(fi)
    
    RX = np.array([[1, 0,      0],
                   [0, cos_fi, -sin_fi],
                   [0, sin_fi,  cos_fi]])
    
    cos_teta = math.cos(teta)
    sin_teta = math.sin(teta)
    
    RY = np.array([[cos_teta, 0, sin_teta],
                  [0        , 1, 0],
                  [-sin_teta, 0, cos_teta]])
    
    cos_ksi = math.cos(ksi)
    sin_ksi = math.sin(ksi)
    RZ = np.array([[cos_ksi, -sin_ksi, 0],
                   [sin_ksi, cos_ksi,  0],
                   [0      , 0,        1]])
    
    A = np.dot(RZ,RY)
    A = np.dot(A,RX)
    return A

def a2Euler(A):
    if A[2,0] == -1:
        fi = 0
        teta = np.pi / 2
        ksi = -2*np.arctan2(A[1,1], -A[0,1])
    elif A[2,0] < 1:
        fi = np.arctan2( A[2,1], A[2,2])
        teta = np.arcsin(-A[2,0])
        ksi = np.arctan2(A[1,0], A[0,0])
    elif A[2,0] == 1:
        fi = 0
        teta = -np.pi / 2
        ksi = np.arctan2(A[0,1], -A[1,1])
    else:
        print("uneta losa matrica A")
        return (0,0,0)
    
    return (fi, teta, ksi)

def axisAngle(A):
    return 0

def rodrigez(p, fi):
    q = math.sqrt(p[0]**2 + p[1] ** 2 + p[2]**2)
    p = p * 1/q
    
    px = np.array([[0,    -p[2], p[1]], 
                   [p[2],  0,    -p[0]], 
                   [-p[1], p[0], 0]])
    
    A = np.sin(fi) * px
    A = A + np.eye(3) * math.cos(fi)
    
    p = p.reshape(1,3)
    pT = p.reshape(3,1)     
    A = A + (1 - math.cos(fi)) * pT.dot(p)
    
    return A

def axisAngle2q(p, fi):
    d = math.sqrt(p[0]**2 + p[1]**2 + p[2]**2)
    p = p * 1/d
    p = np.sin(fi/2) * p
    
    return (p[0], p[1], p[2], np.cos(fi/2));

def q2AxisAngle(q): 
    a,b,c,d = q
    p = np.array([a,b,c])
    
    if d < 0:
        p *= -1
        d *= -1
    
    if d == 1:
        p = (1,0,0)
    else:
        s = math.sqrt(p[0]**2 + p[1]**2 + p[2]**2)
        p = p * 1/s
    
    return (p ,2 * np.arccos(d))

def normalizuj(q):
    suma = np.sum(q)
    
    return q * 1/suma
    


In [4]:
# ulaz 2
fi2 = np.arcsin(math.sqrt(6)/4)
teta2 = np.arctan2(1,3)
ksi2 = np.arctan2(math.sqrt(6), 2)

A2 = np.array([[3,1,math.sqrt(6)],
               [1,3,-math.sqrt(6)],
               [-math.sqrt(6),math.sqrt(6),2]]) * 1/4

p2 = np.array([1/math.sqrt(2), 1/math.sqrt(2), 0])
alfa2 = np.pi/3


# Test euler2A
Ar = euler2A(fi2, teta2, ksi2)
print('Euler2a:')
#print(Ar)
print(A2)

# Test a2Euler
(fir, tetar, ksir) = a2Euler(Ar)
print("a2Euler")
print('old: ', fi2, teta2, ksi2)
print('new: ', fir, tetar, ksir)

# Test axisAngle
#axisAngle(A)

# Test rodrigez
Ar = rodrigez(p2 , alfa2)
print("rodrigez:")
print(Ar)
print(A2)

# Test axisAngle2q
qr = axisAngle2q(p2, alfa2)
print('axisAngle2q:')
print(qr)

# Normalizacija
#qr = normalizuj(qr)
print(qr)

# Test q2AxisAngle
(pr, fir) = q2AxisAngle(qr)
print('q2AxisAngle:')
print(pr, fir)

Euler2a:
[[ 0.75        0.25        0.61237244]
 [ 0.25        0.75       -0.61237244]
 [-0.61237244  0.61237244  0.5       ]]
a2Euler
old:  0.6590580358264089 0.3217505543966422 0.8860771237926136
new:  0.6590580358264089 0.3217505543966422 0.8860771237926136
rodrigez:
[[ 0.75        0.25        0.61237244]
 [ 0.25        0.75       -0.61237244]
 [-0.61237244  0.61237244  0.5       ]]
[[ 0.75        0.25        0.61237244]
 [ 0.25        0.75       -0.61237244]
 [-0.61237244  0.61237244  0.5       ]]
axisAngle2q:
(0.35355339059327373, 0.35355339059327373, 0.0, 0.8660254037844387)
(0.35355339059327373, 0.35355339059327373, 0.0, 0.8660254037844387)
q2AxisAngle:
[0.70710678 0.70710678 0.        ] 1.0471975511965974


In [53]:
# ulaz 1
fi1 = 0
teta1 = np.pi/2
ksi1 = np.pi

A1 = np.array([[0,0,-1],[0,-1,0],[-1,0,0]])

p1 = np.array([-1/math.sqrt(2), 0, 1/math.sqrt(2)])
alfa1 = np.pi


# Test euler2A
Ar = euler2A(fi1, teta1, ksi1)
print('Euler2a:')
print(Ar)
print(A1)

# Test a2Euler
(fir, tetar, ksir) = a2Euler(Ar)
print("a2Euler")
print('old: ', fi1, teta1, ksi1)
print('new: ', fir, tetar, ksir)

# Test axisAngle
#axisAngle(A)

# Test rodrigez
Ar = rodrigez(p1 , alfa1)
print("rodrigez:")
print(Ar)
print(A1)

# Test axisAngle2q
qr = axisAngle2q(p1, alfa1)
print('axisAngle2q:')
print(qr)

# Normalizacija
#qr = normalizuj(qr)
print(qr)

# Test q2AxisAngle
(pr, fir) = q2AxisAngle(qr)
print('q2AxisAngle:')
print(pr, fir)

Euler2a:
[[-6.12323400e-17 -1.22464680e-16 -1.00000000e+00]
 [ 7.49879891e-33 -1.00000000e+00  1.22464680e-16]
 [-1.00000000e+00  0.00000000e+00  6.12323400e-17]]
[[ 0  0 -1]
 [ 0 -1  0]
 [-1  0  0]]
a2Euler
old:  0 1.5707963267948966 3.141592653589793
new:  0 1.5707963267948966 3.141592653589793
rodrigez:
[[ 2.22044605e-16 -8.65956056e-17 -1.00000000e+00]
 [ 8.65956056e-17 -1.00000000e+00  8.65956056e-17]
 [-1.00000000e+00 -8.65956056e-17  2.22044605e-16]]
[[ 0  0 -1]
 [ 0 -1  0]
 [-1  0  0]]
axisAngle2q:
(-0.7071067811865476, 0.0, 0.7071067811865476, 6.123233995736766e-17)
(-0.7071067811865476, 0.0, 0.7071067811865476, 6.123233995736766e-17)
q2AxisAngle:
[-0.70710678  0.          0.70710678] 3.141592653589793
